<a href="https://colab.research.google.com/github/batiukmaks/NLP-Labs/blob/main/labs/LPNLP_08_Text_generation_(2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерація тексту

У цій роботі ми використаємо мовну модель для генерації тексту.

У класичної мовної моделі є два взаємопов'язані визначення:

1. Оцінити ймовірність вхідного тексту.
2. Видати ймовірнісний розподіл наступного слова для даного префіксу.

Для генерації тексту нам ідеально підходить друге визначення.

## Початок роботи

Будь ласка, заповніть поля `EMAIL`, `NAME` та `GROUP` нижче:

In [1]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 4.2 MB/s eta 0:00:00


In [3]:
################################################################################
# FILL-IN:
#-----------------------------------------------------------------------
EMAIL = "maksym.batiuk.kn.2021@lpnu.ua"  # заповніть вашим значенням
################################################################################

import lpnlp

lab = lpnlp.start(email=EMAIL, lab="text_generation")

Удачі!


### Завантаження моделі

Тренування мовної моделі з нуля займає багато часу: від кількох годин або днів для маленьких та середніх моделей й аж до кількох місяців чи навіть років для великих. Звичайно, за рахунок розпаралелювання тренування на багатьох GPU, реальний час рідко буває більше місяця-двох.

Для цієї роботи (як і в реальному житті), ми візьмемо претреновану модель. Хтось натренував її за нас. В данному випадку, візьмемо GPT-2 від компанії OpenAI.

Ця модель, як і безліч інших, зберігається на [HuggingFace Models](https://huggingface.co/models). Завантажити та працювати з нею зручно через бібліотеку [HuggingFace Transformers](https://github.com/huggingface/transformers/)

In [4]:
!pip install transformers

In [5]:
import transformers
import torch

In [6]:
model = transformers.AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Токенізація

Кожна модель має словник токенів, з яким вона тренувалася, та правила токенізації (розбиття тексту на токени). Обов'язково слід використовувати той самий словник та метод.

Бібліотека transformers вміє робити це для кожної підтримуваної моделі.

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Перевіримо токенізатор:

In [8]:
input_text = "Hello, LP NLP!"
token_ids = tokenizer.encode(input_text)
token_ids

[15496, 11, 18470, 399, 19930, 0]

In [9]:
tokenizer.convert_ids_to_tokens(token_ids)

['Hello', ',', 'ĠLP', 'ĠN', 'LP', '!']

Рідкісні слова будуть розбиті на підслова:

In [10]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("My name is Oleksiy Syvokon"))

['My', 'Ġname', 'Ġis', 'ĠOle', 'ks', 'iy', 'ĠSy', 'v', 'ok', 'on']

## Перевірка моделі -- один крок ітерації

Ми генеруватимемо текст в циклі токен за токеном, зліва направо. Але для початку розберімо, як виглядає один крок такого циклу.


На кожному кроці на вхід моделі подаємо ті токени, які вже згенеровано.

На першому кроці у нас ще нічого не згенеровано. Тому починаємо зі токену `<BOS>` ("begin of sentence"). В різних моделях він виглядає по-різному. Подивимося, що в GPT-2:


In [11]:
tokenizer.bos_token

'<|endoftext|>'

In [12]:
tokenizer.bos_token_id

50256

In [13]:
# Формуємо вхідний батч. Майже завжди для більшої ефективності
# нейронні мережі очікують на вхід кілька незалежних речень
# (або зображень у випадку з комп'ютерним зором)
#
# В нашій роботі ми завжди працюємо лише з одним реченням,
# тож розмір батча дорівнює одинці. Але все одно маємо
# оформити вхід як матрицю:
input_ = torch.LongTensor([[tokenizer.bos_token_id]])
input_

tensor([[50256]])

In [14]:
# Нарешті робимо крок генерації
output = model(input_, return_dict=True)

# На виході модель повертає вектор з logits -- ненормалізованими
# ймовірностями кожного токена в словнику
output.logits.shape

torch.Size([1, 1, 50257])

Чому маємо саме таку розмірність?

* Перша одиниця -- розмір батча, тобто кількість вхідних речень.
* Друга одиниця -- це довжина вхідної послідовності. На першому кроці ми подали лише один токен (bos_token)
* 50257 -- це розмір словника

In [15]:
tokenizer.vocab_size

50257

Поки що модель повернула не ймовірності, а просто якісь числа. Їм треба нормалізувати функцією softmax:

In [16]:
probs = torch.softmax(output.logits, dim=-1)

In [17]:
# Розмірність має збігатися з розміром словника
probs.shape

torch.Size([1, 1, 50257])

In [18]:
# Сума ймовірностей має дорівнювати 1.0
probs.sum()

tensor(1.0000, grad_fn=<SumBackward0>)

Ймовірності можуть ставати дуже маленькими та причиняти проблеми у зв'язку з обмеженою точністю float чисел. Тому прийнято працювати з логарифмами ймовірностей.

In [19]:
log_probs = torch.log_softmax(output.logits, dim=-1)

# Щоб перейти до звичайних ймовірностей, маємо зробити експоненціювання
log_probs.exp().sum()

tensor(1.0000, grad_fn=<SumBackward0>)

Кожному слову в словнику відповідає своя ймовірність бути побаченим
після заданого префікса. Префіксом у нас поки що був лише одни `bos_token`.


In [20]:
# Яка ймовірність, що речення почнеться зі слів "I", "red", "Why"?
for word in ("I", "red", "Why"):
    index = tokenizer.convert_tokens_to_ids(word)
    prob = log_probs[0, 0, index].exp()
    print(f"P({word}) = {prob}")

P(I) = 0.018320858478546143
P(red) = 0.00010652346827555448
P(Why) = 0.0008686637738719583


Тепер ми маємо ймовірностний розподіл по словнику. Можемо обрати слово, яке вважатимемо згенерованим. Тут можливі кілька стратегій, які ми розглянемо в наступних розділах.

## Greedy decoding

Найпростіший (але й не дуже цікавий) спосіб -- це завжди обирати токен з найбільшою ймовірністю:

In [21]:
next_token_id = log_probs.argmax()
next_token_id

tensor(198)

In [22]:
probs[-1, -1, 198]

tensor(0.0623, grad_fn=<SelectBackward0>)

In [23]:
tokenizer.convert_ids_to_tokens([next_token_id])  # Наш перший згенерований токен

['Ċ']

Зберемо код докупи та додамо цикл. В циклі ми продовжуватимемо генерувати текст токен за токеном, поки не настане одна з двох умов:
1. Модель видала спецальний токен `eos_token` (end of sentence)
2. Довжина згенерованого тексту перевищила певний поріг `max_len`

У хорошої моделі в більшості випадків має спрацьовувати перша умова зупинки. Проте іноді модель може впасти в безкінчений цикл. Щоб цьому запобігти, маємо другу умову.

In [24]:
@torch.no_grad()
def greedy_decode(model, tokenizer, max_len=50):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = probs.argmax()

        # Зупиняємося, якщо досягли кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    return result


generated_token_ids = greedy_decode(model, tokenizer)

Маємо список згенерованих індексів токенів, який починається ось так:

In [25]:
generated_token_ids[:10]

[50256, 198, 464, 717, 640, 314, 2497, 262, 649, 2196]

Перетворимо їх в текст:

In [26]:
tokens = tokenizer.convert_ids_to_tokens(generated_token_ids)
tokens[:10]

['<|endoftext|>',
 'Ċ',
 'The',
 'Ġfirst',
 'Ġtime',
 'ĠI',
 'Ġsaw',
 'Ġthe',
 'Ġnew',
 'Ġversion']

In [27]:
generated_text = "".join(tokens)
generated_text

'<|endoftext|>ĊTheĠfirstĠtimeĠIĠsawĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcited.ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠto'

In [28]:
lab.checkpoint("greedy decode", generated_text)

Відповідь правильна ✅



'<|endoftext|>ĊTheĠfirstĠtimeĠIĠsawĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcited.ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠto'

### Примітка: Byte-pair encoding (BPE)

Наша модель використовує subword токенізацію, а саме byte-pair encoding (BPE). В сучасному NLP це найрозповсюдженіший спосіб токенізації. Детально можете подивитися в [цьому відео](https://www.youtube.com/watch?v=tOMjTCO0htA).

Для наших цілей зараз важливо, що BPE заміняє пробіли на спеціальні Unicode-символи "Ġ". Серед інших моделей широко поширений варіант "▁" (зверніть увагу, це не звичайний символ підкреслення "_"). Щоб отримати чистий текст, треба виконати наступну заміну:

In [29]:
def bpe_decode(s):
    result = s.replace("Ġ", " ")
    result = result.replace("Ċ", "\n")
    return result

bpe_decode(generated_text)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

Але краще довіритися токенізатору й зробити цю роботу за нас:

In [30]:
tokenizer.convert_tokens_to_string(tokens)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

## Generic decoding function

Обирати слово з найбільшою ймовірністю -- не найкращий варіант для генерації тексту хоча б тому, що він завжди детерміновано призводить до однієї послідовності. Нижче ми подивимося на цікавіші альтернативи.

Цикл генерації залишиться той самий, що і в `greedy_decode()`. Відрізнятися буде лише один рядок -- той, в якому ми приймали рішення, яке слово обрати. Для зручності, винесемо цей рядок в окрему функцію. Ця функція прийматиме на вхід ймовірностний розподіл по словнику і повертає обраний токен.

In [31]:
def greedy_choice(probs):
    return probs.argmax()

Функцію генерації також трохи переробимо.

По-перше, додамо параметр `sample_fn` -- це має бути функція, яка обирає слово з ймовірностного розподілу, наприклад, `greedy_choice`.

По-друге, для зручності виконуватимемо BPE декодинг у середині функції генерації.

In [32]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, max_len=50, bpe_decode=True):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(log_probs.exp())     # <---------------- цей рядок змінено

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result


generate(model, tokenizer, greedy_choice)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

## Simple sampling

Перший альтернатива -- це sampling. Тепер ми обираємо наступний токен випадково, але пропорційно до ймовірностей.

In [33]:
def simple_sample(probs):
    return torch.multinomial(probs, num_samples=1)[0]

# Згенеруємо 5 речень
for i in range(1, 6):
    result = generate(model, tokenizer, simple_sample)
    print(f"#{i}: {result}")
    print()

#1: <|endoftext|>Enthusiasts occupy three of our summer's longest hours in the sun, and their energy-products are pruned to about a small hour for the hunt and draw in brisk Energy.

Hello References

Meeting and Promising

#2: <|endoftext|>iffwheel(4, 2): restoration#128}*1*elemental0))))*3 - ( novice oil)


* msg(&i); html += '</script id=' + parseInt(173, 34) * h

#3: <|endoftext|>Reportedly sporting the likes of XO Sneely to his left, Spooky headstrong James Gordon should be a bit difficult to knock off this year, as he's with the speedster so incessantly at home, and he's

#4: <|endoftext|>**ALLVA NOVA ***

#5: <|endoftext|>It maneuvers to funnel an insectoid that resembles an egg on a specimen's head. Its mouth gives off an excited rush when pulled through the ear canal. Malasseldzius, pronounced "Malasseld", is a species of Latin ten



## Sampling with temperature

Ми також можемо впливати на генерацію параметром температури softmax.

Більші значення температури призводять до того, що різниця між ймовірностями токенів зменшується, тобто розподіл стає більш рівномірним. На практиці це означає, що менш ймовірні варіанти обиратимуться частіше і згенерований текст може бути цікавішим. Однак якщо продовжувати піднімати температуру, то текст спочатку втратить зв'язність, далі почнуть розпадатися слова та граматичність.

Менші значення температури змінюють розподіл таким чином, що основна ймовірніста маса припадає на невелику кількість топових токенів. При температурі 0 вся ймовірність дістанеться одному токену й семплінг перетвориться на greedy decoding.

Згенеруємо тексти з різною температурою:

In [34]:
for temperature in (0.1, 0.3, 0.5, 0.8, 1.0, 1.25, 1.5, 2.0, 3.0, 5.0):

    def sample_with_temp(probs):
        updated_probs = probs.log().div(temperature).exp()
        return simple_sample(updated_probs)

    print(f"Sampling with temperature={temperature}")
    result = generate(model, tokenizer, sample_with_temp, max_len=15)
    print(result)
    print()

Sampling with temperature=0.1
<|endoftext|>
The first time I saw the new version of the game, I

Sampling with temperature=0.3
<|endoftext|>
The first time I saw a video of the new Apple Watch was

Sampling with temperature=0.5
<|endoftext|>The cyberattack on Sony Pictures Entertainment's website was disclosed by a third

Sampling with temperature=0.8
<|endoftext|>Our past experience as executives and studio executives at Sony Pictures has been to

Sampling with temperature=1.0
<|endoftext|>See better

Share this: Facebook

Twitter

Google

Sampling with temperature=1.25
<|endoftext|>Len Lowry The Sect beast LanceMymah Spectre Brierozzle


Sampling with temperature=1.5
<|endoftext|> would quiet writing ads trick great like novel thriller Australia promises potential world 113

Sampling with temperature=2.0
<|endoftext|>Doyal Information Servcollege Facts Drops Sour ??field Bears Knotmiegger

Sampling with temperature=3.0
<|endoftext|>Got server sublime======== bypass riot ¶ings++++ eas

In [35]:
# Яке значення `temperature` здається вам оптимальною?
lab.checkpoint("softmax temperature", 1.0)

Відповідь правильна ✅
Окей, добре


1.0

## Top-k sampling

In [36]:
def top_k_sampling(probs, k):
    topk = probs.topk(k)
    index = torch.multinomial(topk.values, num_samples=1)[0]
#     print(f"Top {k} words take {topk.values.sum():%} probability mass")
    return topk.indices[index]

In [37]:
k = 15
sample_fn = lambda probs: top_k_sampling(probs, k=k)
for i in range(1, 6):
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{i}: {result}")
    print()

#1: <|endoftext|>In the first round of the NBA playoffs, a team has to overcome the odds, and it

#2: <|endoftext|>We don't know much more about the state of the game right now, and the only information

#3: <|endoftext|>The government has also been accused of using its power to impose "anti-nationalist" sanctions

#4: <|endoftext|>The U.S. Department of Agriculture issued a warning to livestock producers in January, stating that

#5: <|endoftext|>
In the wake of Hurricane Harvey, which killed at least 30 people, the National Weather Service



## Nucleus (top-p) sampling

In [38]:
def nucleus_sampling(probs, max_p):
    sorted_probs = probs.sort(descending=True)
    cum_prob = 0.0
    sample_indices = []
    sample_probs = []
    for i in range(0, len(sorted_probs.values)):
        p = sorted_probs.values[i]
        cum_prob += p
        sample_probs.append(p)
        sample_indices.append(sorted_probs.indices[i])
        if cum_prob >= max_p:
            break

    index = torch.multinomial(torch.tensor(sample_probs), num_samples=1)
    return sample_indices[index]

In [39]:
for max_p in (0.0, 0.1, 0.3, 0.5, 0.6, 0.8, 1.0):
    sample_fn = lambda probs: nucleus_sampling(probs, max_p=max_p)
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{max_p}: {result}")
    print()

#0.0: <|endoftext|>
The first time I saw the new version of the game, I was so excited. I

#0.1: <|endoftext|>"I'm not going to be a racist," he said. "I'm not going to

#0.3: <|endoftext|>A former FBI agent has been charged with stealing $5 million from the bank of a suspected drug

#0.5: <|endoftext|>
A new study has found that more than a third of the 5,800 people in the

#0.6: <|endoftext|>One of the main features of the SLC is its ability to use its own filtering software,

#0.8: <|endoftext|>b98feb4cc80fd70a5e4af82b97b0

#1.0: <|endoftext|>hup for penis dye? Nut Paleo What's a woman feeling after the removal of their oral



## Start from prompt

До цього моменту ми генерували текст з нуля. Однак значно кориснішим є задача генерації тексту від певного префікса або "підказки" -- в англійській мові це називається "prompt".

Prompt дозволить нам контролювати тематику згенерованого тексту або, як ми побачимо на іншій лекції, допоможе моделі виконувати різноманітні завдання.

In [40]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, prompt, max_len=50, bpe_decode=True):

    result = tokenizer.encode(prompt)     # <---- цей рядок додано

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(log_probs.exp())

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result

In [41]:
k = 3
sample_fn = lambda probs: top_k_sampling(probs, k=k)
for i in range(1, 5):
    result = generate(model, tokenizer, sample_fn, prompt="After doing the workout I")
    print(f"#{i}: {result}")
    print()


# Спробуйте змінити prompt на щось інше

#1: After doing the workout I noticed a slight increase in the number of muscles in my back. I also noticed a slight reduction in my waist and hips. I also noticed that my shoulders were slightly less relaxed.

I'm not sure what the cause

#2: After doing the workout I was able to get some of the muscles back and get some of the muscles back in place. I also got some of the muscles back in the way I needed them to, and it worked great."

#3: After doing the workout I was surprised by how much I felt. I felt like a little bit of a freak out, but I was also able to feel my muscles. I felt like I was in a really strong physical condition, and I was feeling

#4: After doing the workout I was able to see that the muscles in my body are not as weak as they used to be and I am able to move more easily and I am not afraid to get hurt.

I also feel better after doing a



# Zero-shot класифікація

Ми будемо досліджувати цю тему на наступній лекції. Але спробуємо застосувати мовну модель для задачі класифікації вже зараз. Хоча наша модель надто маленька й слабка для серйозного використання в zero-shot.


In [42]:
generate(model, tokenizer, greedy_choice, prompt="This film is awful! Actor play is terrible. The plot is dull. I would rate it as a").splitlines()[0]


'This film is awful! Actor play is terrible. The plot is dull. I would rate it as a B+.'

In [43]:
generate(model, tokenizer, greedy_choice, prompt="This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a").splitlines()[0]


'This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a 5.'

In [44]:
lab.answer("Готово!")

Відповідь правильна ✅
Ця робота не має однієї правильної відповіді. Вважаємо лабу пройденою :) Формочка: https://tally.so/r/mZ81ve


'Готово!'

# Real world

У цій роботі ми імплементували кілька методів декодінгу. Але, звичайно, все вже зроблено за нас.

Подивіться на параметри функції [generate()](https://huggingface.co/docs/transformers/v4.21.1/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate) з бібліотеки transformers. Багато з них мають виглядати знайомими.

Повний приклад використання:

In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

prompt = "I hope that in that practical class on text generation you"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, do_sample=True, top_p=0.8, max_length=50)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['I hope that in that practical class on text generation you will be able to see that you will be able to learn how to use the tools we provide.\n\nThe second stage of this class will take you through the fundamentals of data structures, such']

# The End


In [46]:
# Сподіваюсь, ця лаба була не надто важкою :)
lab.answer("Готово!")

Відповідь правильна ✅
Ця робота не має однієї правильної відповіді. Вважаємо лабу пройденою :) Формочка: https://tally.so/r/mZ81ve


'Готово!'